# Panic Project (DHLAB) - Data Analysis

author:  `@cyshin971`  

date:    `2025-07-14`  

Instructions:
- Run `data_preprocessing.ipynb`
- Output can be found in `./data/`

version: `3.1`

> version `1.1`: preprocessing separated to `data_preprocessing.ipynb` (version `1.0`)  
> version `2.0`: Updated to consensus on progress meeting `20250619`  
> version `3.0`: Release Version  
> -  version `3.1`: Directory organization

In [ ]:
version = '3-1'

# 📚 | Import Libraries 

- `python` (`3.10`)
- `pandas`  
- `numpy`
- `json`
- `matplotlib`

In [ ]:
import config as cfg
import logging

import matplotlib.pyplot as plt
logging.getLogger('matplotlib').setLevel(logging.WARNING)

from library.pandas_utils import move_column, aggregate_by_column, create_empty_df, read_csv
from library.text_utils import save_as_csv
from library.json_utils import save_dict_to_file, load_dict_from_file
from library.path_utils import get_file_path

# ⚙️ | Settings

In [ ]:
is_dev = False  # Set to True (Default: False) if you want to run the notebook in development mode
manual_scraped_data_filename = None # Keep as None if you don't want to manually specify a file

In [ ]:
logging.info(f"Notebook running with:\n" +
             f"mode: {'development' if is_dev else 'production'}\n" +
             f"manual_scraped_data_filename: {manual_scraped_data_filename}\n")

# 📁 | Path Variables 

In [ ]:
DATA_PATH = "./data"
TMP_PATH = "./_tmp"
OUT_PATH = TMP_PATH
if is_dev: OUT_PATH = "./_output"
file_desc = ""

try:
	current_config = load_dict_from_file(OUT_PATH, 'current_config')
except FileNotFoundError:
    raise FileNotFoundError(f"File not found: {get_file_path(OUT_PATH, 'current_config.csv')}.\nPlease run data_preprocessing.ipynb first.")

print(f"Loaded current config with {len(current_config)} keys:")
scraped_data_filename = None
for k, v in current_config.items():
    if k == 'scraped_data_filename':
        scraped_data_filename = v
    elif k == 'preproc_version':
        preproc_version = v
    print(f"  {k}: {v}")

if is_dev and manual_scraped_data_filename is not None:
	manual_scraped_data_filename = manual_scraped_data_filename.replace('.csv', '')  # Remove .csv if present
	if not get_file_path(OUT_PATH, f"{manual_scraped_data_filename}").is_dir():
		raise FileNotFoundError(f"Manually specified scraped data directory '{manual_scraped_data_filename}' does not exist in {OUT_PATH}.")
	logging.warning(f"Using manually specified scraped_data_filename: {manual_scraped_data_filename}")
	scraped_data_filename = manual_scraped_data_filename

if scraped_data_filename is None:
	raise ValueError("scraped_data_filename not found in current_config. Please ensure that data_preprocessing.ipynb has been run successfully before running this notebook.")

if is_dev:
    OUTPUT_FILE_PATH = f"{OUT_PATH}/{scraped_data_filename}"
    PREPROC_PATH = f"{OUTPUT_FILE_PATH}/preprocessed"
    OUTPUT_PATH = f"{OUTPUT_FILE_PATH}/analysis"
    file_desc = f"_{version}({scraped_data_filename})"
else:
    OUTPUT_FILE_PATH = DATA_PATH
    PREPROC_PATH = DATA_PATH
    OUTPUT_PATH = f"{DATA_PATH}/analysis"

# ⚒️ | Preprocessed Data

### Load Features

In [ ]:
try:
	features_dict = load_dict_from_file(OUTPUT_FILE_PATH, 'panic_features_dict')
except FileNotFoundError:
    raise FileNotFoundError(f"File not found: {get_file_path(OUT_PATH, 'panic_features_dict.csv')}.\nPlease run data_preprocessing.ipynb first.")

print(f"Loaded features dict with {len(features_dict)} keys:")
for k, v in features_dict.items():
        print(f"  {k}: {v}")

## Load Preprocessed Data

In [ ]:
pre_data = read_csv(get_file_path(PREPROC_PATH, f'panic_pre_data{file_desc}.csv'))
display(pre_data.head(2))
metadata = read_csv(get_file_path(PREPROC_PATH, f'panic_metadata{file_desc}.csv'))
display(metadata.head(2))
demography_data = read_csv(get_file_path(PREPROC_PATH, f'panic_demography_data{file_desc}.csv'))
display(demography_data.head(2))

# 🔍 | Data Analysis

## 🕳️ | Null Value Analysis

Produces csv file of the percentage of the null values for each feature in Daily Log, and Life Log

In [ ]:
null_value_analysis = create_empty_df()
null_value_analysis = pre_data[features_dict['lifelog']+features_dict['dailylog']].isnull().sum().reset_index()
null_value_analysis.rename(columns={0: 'null_count'}, inplace=True)
null_value_analysis.rename(columns={'index': 'feature'}, inplace=True)

# add a column for the data group using the feqtures_dict
null_value_analysis['data_group'] = null_value_analysis['feature'].apply(
    lambda x: 'mood' if x in features_dict['mood'] else
			  'dailylog_life' if x in features_dict['dailylog_life'] else
			  'lifelog_HR' if x in features_dict['lifelog_HR'] else
			  'lifelog_steps' if x in features_dict['lifelog_steps'] else
			  'lifelog_sleep' if x in features_dict['lifelog_sleep'] else
			  'other'
)

# add column for total count
null_value_analysis['total_count'] = len(pre_data)
# find the existing percentage
null_value_analysis['null%'] = (null_value_analysis['null_count'] / null_value_analysis['total_count']) * 100

move_column(null_value_analysis, 'data_group', 1)
move_column(null_value_analysis, 'null%', 2)

save_as_csv(null_value_analysis, OUTPUT_PATH, f"panic_null_value_analysis{file_desc}")

display(null_value_analysis.head(5))

In [ ]:
plt.figure(figsize=(5, 10))  # Adjusted to make it thinner horizontally and longer vertically
colors = ['skyblue'] * len(null_value_analysis)
ax = null_value_analysis.sort_values('null%', ascending=True).plot(
	kind='barh', x='feature', y='null%', color=colors, legend=False, width=0.5  # Reduced bar width to increase gaps
)
plt.title("Percentage of Null Values by Feature")
plt.xlabel("Percentage of Null Values (%)")
plt.xlim(0, 100)  # Set the x-axis limit to 0-100
# get rid of the right and top spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Adjust y-axis labels for better visibility
ax.set_yticklabels(ax.get_yticklabels(), fontsize=5, rotation=0, ha='right')

# Add labels with values to the right of each bar
for p in ax.patches:
	value = f"{p.get_width():.2f}%"
	ax.annotate(value,
				(p.get_width() + 1, p.get_y() + p.get_height() / 2),
				ha='left', va='center', fontsize=5, color='black', xytext=(0, 0),
				textcoords='offset points')

plt.show()

## 🤯 | Patient-level Analysis

**Description**
- `n_entries`: Number of entries per patient
- `n_valid_3_entries`: Number of valid (3 prior consecutive days of data) entries per patient
- `n_valid_2_entries`: Number of valid (2 prior consecutive days of data) entries per patient
- `n_valid_1_entries`: Number of valid (1 prior consecutive days of data) entries per patient
- `n_panic`: Number of panic events per patient
- `max_severity`: Maximum panic severity experienced by patient
- `min_severity`: Minimum panic severity experienced by patient
- `mean_severity`: Average panic severity experienced by patient
- `n_dailylog`: Number of daily log entries per patient
- `n_lifelog`: Number of life log entries per patient
- `n_questionnaire`: Number of questionnaire entries per patient
- `sum_dtype`: Number of data (by group) per patient
- `mean_dtype`: Average number of data types per entry per patient

In [ ]:
agg_matrix = [
	('n_entries', 'entry_id', 'count'),
	('n_valid_3_entries', 'valid_entry_3', 'sum'),
	('n_valid_2_entries', 'valid_entry_2', 'sum'),
	('n_valid_1_entries', 'valid_entry_1', 'sum'),
	('n_panic', 'panic_label', 'sum'),
    ('max_severity', 'severity', 'max'),
	('min_severity', 'severity', 'min'),
	('mean_severity', 'severity', 'mean'),
	('n_dailylog', 'dailylog_data', 'sum'),
	('n_lifelog', 'lifelog_data', 'sum'),
	('n_questionnaire', 'questionnaire_data', 'sum'),
	('sum_dtype', 'dtype_n', 'sum'),
	('mean_dtype', 'dtype_n', 'mean'),
]
patient_analysis_data = create_empty_df()
patient_analysis_data = aggregate_by_column(metadata, 'ID', agg_matrix)

display(patient_analysis_data.head(5))
save_as_csv(patient_analysis_data, OUTPUT_PATH, f"panic_patient_analysis{file_desc}")

## 🤯🕳️ | Patient-level Null Value Analysis

Produces csv file of the percentage of the Null values for each feature in Daily Log, and Life Log for each patient

In [ ]:
agg_matrix = [('n_entries', 'entry_id', 'count')]
cols_to_analyze = features_dict['lifelog'] + features_dict['dailylog']

for col in cols_to_analyze:
    agg_matrix.append((col, col, 'count'))

patient_null_analysis_data = create_empty_df()
patient_null_analysis_data = aggregate_by_column(pre_data, 'ID', agg_matrix)

for col in cols_to_analyze:
    patient_null_analysis_data[col] = (patient_null_analysis_data['n_entries'] - patient_null_analysis_data[col]) / patient_null_analysis_data['n_entries'] * 100

patient_null_analysis_data['lifelog_avg'] = patient_null_analysis_data[features_dict['lifelog']].mean(axis=1)
patient_null_analysis_data['lifelog_HR_avg'] = patient_null_analysis_data[features_dict['lifelog_HR']].mean(axis=1)
patient_null_analysis_data['lifelog_steps_avg'] = patient_null_analysis_data[features_dict['lifelog_steps']].mean(axis=1)
patient_null_analysis_data['lifelog_sleep_avg'] = patient_null_analysis_data[features_dict['lifelog_sleep']].mean(axis=1)

patient_null_analysis_data['dailylog_avg_avg'] = patient_null_analysis_data[features_dict['dailylog']].mean(axis=1)
patient_null_analysis_data['dailylog_mood_avg'] = patient_null_analysis_data[features_dict['mood']].mean(axis=1)
patient_null_analysis_data['dailylog_life_avg'] = patient_null_analysis_data[features_dict['dailylog_life']].mean(axis=1)

patient_null_analysis_data['total_avg'] = patient_null_analysis_data[features_dict['lifelog'] + features_dict['dailylog']].mean(axis=1)

display(patient_null_analysis_data.head(5))
save_as_csv(patient_null_analysis_data, OUTPUT_PATH, f"panic_patient_null_analysis{file_desc}")

# 📒 | Reports

In [ ]:
from library.matplotlib_utils import plot_histogram_of_counts

## Overview

In [ ]:
unique_ids =  pre_data['ID'].unique()
print(f"Total number of unique IDs in pre_data_filled: {len(unique_ids)}")
print(f"Total number of entries in pre_data_filled: {len(pre_data)}")
panic_entries = pre_data[pre_data['panic_label'] == 1]
print(f"Total number of panic entries in pre_data_filled: {len(panic_entries)}")

In [ ]:
filled_entry_ids = pre_data['entry_id'].unique()
print(f"Total number of daily log entries: {metadata[metadata['dailylog_data'] == 1].shape[0]} / {len(filled_entry_ids)} ({metadata[metadata['dailylog_data'] == 1].shape[0] / len(filled_entry_ids) * 100:.2f}%)")
print(f"Total number of life log entries: {metadata[metadata['lifelog_data'] == 1].shape[0]} / {len(filled_entry_ids)} ({metadata[metadata['lifelog_data'] == 1].shape[0] / len(filled_entry_ids) * 100:.2f}%)")
print(f"Total number of questionnaire entries: {metadata[metadata['questionnaire_data'] == 1].shape[0]} / {len(filled_entry_ids)} ({metadata[metadata['questionnaire_data'] == 1].shape[0] / len(filled_entry_ids) * 100:.2f}%)")

In [ ]:
panic_patients = metadata[metadata['panic_label'] == 1]['ID'].unique()
print(f"Total number of patients with panic events: {len(panic_patients)}")

## 🗒️ | Entry-level Analysis

### Data Groups

In [ ]:
plot_histogram_of_counts(metadata['dtype_n'], title='Number of data types per entry', xlabel='Number of data types', ylabel='Number of entries', zero_start=False, bins_step=1)
print(f"Number of entries with 1 data type: {metadata[metadata['dtype_n'] == 1].shape[0]} ({metadata[metadata['dtype_n'] == 1].shape[0] / metadata.shape[0] * 100:.2f}%)")
print(f"Number of entries with 2 data types: {metadata[metadata['dtype_n'] == 2].shape[0]} ({metadata[metadata['dtype_n'] == 2].shape[0] / metadata.shape[0] * 100:.2f}%)")
print(f"Number of entries with 3 data types: {metadata[metadata['dtype_n'] == 3].shape[0]} ({metadata[metadata['dtype_n'] == 3].shape[0] / metadata.shape[0] * 100:.2f}%)")

### Valid Entries

In [ ]:
print(f"Total number of valid entries (3 days): {metadata['valid_entry_3'].sum()} ({metadata['valid_entry_3'].mean() * 100:.2f}%)")
print(f"Total number of valid entries (2 days): {metadata['valid_entry_2'].sum()} ({metadata['valid_entry_2'].mean() * 100:.2f}%)")
print(f"Total number of valid entries (1 day): {metadata['valid_entry_1'].sum()} ({metadata['valid_entry_1'].mean() * 100:.2f}%)")

In [ ]:
panic_n = pre_data[pre_data['panic_label'] == 1].shape[0]
print(f"Total number of panic events (panic_label=1): {panic_n}")
valid_panic_events_3 = metadata[(metadata['valid_entry_3'] == 1) & (metadata['dbp'] == 0)].shape[0]
print(f"Total number of valid panic events (n_prior_data >= 3 days): {valid_panic_events_3} ({valid_panic_events_3 / panic_n * 100:.2f}%)")
valid_panic_events_2 = metadata[(metadata['valid_entry_2'] == 1) & (metadata['dbp'] == 0)].shape[0]
print(f"Total number of valid panic events (n_prior_data >= 2 days): {valid_panic_events_2} ({valid_panic_events_2 / panic_n * 100:.2f}%)")
valid_panic_events_1 = metadata[(metadata['valid_entry_1'] == 1) & (metadata['dbp'] == 0)].shape[0]
print(f"Total number of valid panic events (n_prior_data >= 1 day): {valid_panic_events_1} ({valid_panic_events_1 / panic_n * 100:.2f}%)")

In [ ]:
one_dbp = metadata[metadata['dbp'] == 1]
print(f"Total number of entries with dbp=1: {one_dbp.shape[0]} ({one_dbp.shape[0] / metadata.shape[0] * 100:.2f}%)")
two_dvp = metadata[metadata['dbp'] == 2]
print(f"Total number of entries with dbp=2: {two_dvp.shape[0]} ({two_dvp.shape[0] / metadata.shape[0] * 100:.2f}%)")

## 🤢 | Patient-level Analysis

In [ ]:
pre_data_ids = pre_data['ID'].unique()

### Patient-level Valid Entries

In [ ]:
print(f"Number of patients with > 100 entries: {patient_analysis_data[patient_analysis_data['n_entries'] > 100].shape[0]} / {len(pre_data_ids)} ({patient_analysis_data[patient_analysis_data['n_entries'] > 100].shape[0] / len(pre_data_ids) * 100:.2f}%)")
print(f"Number of patients with <= 100 entries: {patient_analysis_data[patient_analysis_data['n_entries'] <= 100].shape[0]} / {len(pre_data_ids)} ({patient_analysis_data[patient_analysis_data['n_entries'] <= 100].shape[0] / len(pre_data_ids) * 100:.2f}%)")
print(f"Number of patients with > 30 entries: {patient_analysis_data[patient_analysis_data['n_entries'] > 30].shape[0]} / {len(pre_data_ids)} ({patient_analysis_data[patient_analysis_data['n_entries'] > 30].shape[0] / len(pre_data_ids) * 100:.2f}%)")
print(f"Number of patients with <= 30 entries: {patient_analysis_data[patient_analysis_data['n_entries'] <= 30].shape[0]} / {len(pre_data_ids)} ({patient_analysis_data[patient_analysis_data['n_entries'] <= 30].shape[0] / len(pre_data_ids) * 100:.2f}%)")
plot_histogram_of_counts(patient_analysis_data['n_entries'], title='Number of Entries per Patient',
                         xlabel='Number of Entries', ylabel='Number of Patients', bins_step=20, exclude_zero=True)
valid_patients = patient_analysis_data[patient_analysis_data['n_valid_1_entries'] > 0]
n_valid_patients_1 = valid_patients.shape[0]
print(f"Number of valid patients (1 days): {valid_patients.shape[0]} / {len(pre_data_ids)}")
print(f"{len(pre_data_ids) - valid_patients.shape[0]} patients do not have valid entries (i.e., no data for at least 1 days before panic event)")
plot_histogram_of_counts(patient_analysis_data['n_valid_1_entries'], title='Number of Valid Entries (1 days) per Patient',
                         xlabel='Number of Valid Entries (1 days)', ylabel='Number of Patients', bins_step=20, exclude_zero=True)
valid_patients = patient_analysis_data[patient_analysis_data['n_valid_2_entries'] > 0]
n_valid_patients_2 = valid_patients.shape[0]
print(f"Number of valid patients (2 days): {valid_patients.shape[0]} / {len(pre_data_ids)}")
print(f"{len(pre_data_ids) - valid_patients.shape[0]} patients do not have valid entries (i.e., no data for at least 2 days before panic event)")
plot_histogram_of_counts(patient_analysis_data['n_valid_2_entries'], title='Number of Valid Entries (2 days) per Patient',
                         xlabel='Number of Valid Entries (2 days)', ylabel='Number of Patients', bins_step=20, exclude_zero=True)
valid_patients = patient_analysis_data[patient_analysis_data['n_valid_3_entries'] > 0]
n_valid_patients_3 = valid_patients.shape[0]
print(f"Number of valid patients (3 days): {valid_patients.shape[0]} / {len(pre_data_ids)}")
print(f"{len(pre_data_ids) - valid_patients.shape[0]} patients do not have valid entries (i.e., no data for at least 3 days before panic event)")
plot_histogram_of_counts(patient_analysis_data['n_valid_3_entries'], title='Number of Valid Entries (3 days) per Patient',
                         xlabel='Number of Valid Entries (3 days)', ylabel='Number of Patients', bins_step=20, exclude_zero=True)
del valid_patients

### Patient-level Panic Analysis

In [ ]:
print(f"Number of patients with > 2 number of panic events: {patient_analysis_data[patient_analysis_data['n_panic'] > 2].shape[0]} / {len(pre_data_ids)} ({patient_analysis_data[patient_analysis_data['n_panic'] > 2].shape[0] / len(pre_data_ids) * 100:.2f}%)")
print(f"Number of patients with <= 2 number of panic events: {patient_analysis_data[patient_analysis_data['n_panic'] <= 2].shape[0]} / {len(pre_data_ids)} ({patient_analysis_data[patient_analysis_data['n_panic'] <= 2].shape[0] / len(pre_data_ids) * 100:.2f}%)")
print(f"Number of patients with > 10 number of panic events: {patient_analysis_data[patient_analysis_data['n_panic'] > 10].shape[0]} / {len(pre_data_ids)} ({patient_analysis_data[patient_analysis_data['n_panic'] > 10].shape[0] / len(pre_data_ids) * 100:.2f}%)")
print(f"Number of patients with <= 10 number of panic events: {patient_analysis_data[patient_analysis_data['n_panic'] <= 10].shape[0]} / {len(pre_data_ids)} ({patient_analysis_data[patient_analysis_data['n_panic'] <= 10].shape[0] / len(pre_data_ids) * 100:.2f}%)")
plot_histogram_of_counts(patient_analysis_data['n_panic'], title='Number of Panic Events per Patient', xlabel='Number of Panic Events', ylabel='Number of Patients', bins_step=5,exclude_zero=False)
print(f"Number of patients with zero panic events: {patient_analysis_data[patient_analysis_data['n_panic'] == 0].shape[0]} / {len(pre_data_ids)}")

In [ ]:
valid_panic_events_patients = patient_analysis_data[(patient_analysis_data['n_panic'] > 0) & (patient_analysis_data['n_valid_3_entries'] > 0)]
print(f"Number of patients with valid panic events (3 days): {valid_panic_events_patients.shape[0]} / {n_valid_patients_3}")
valid_panic_events_patients = patient_analysis_data[(patient_analysis_data['n_panic'] > 0) & (patient_analysis_data['n_valid_2_entries'] > 0)]
print(f"Number of patients with valid panic events (2 days): {valid_panic_events_patients.shape[0]} / {n_valid_patients_2}")
valid_panic_events_patients = patient_analysis_data[(patient_analysis_data['n_panic'] > 0) & (patient_analysis_data['n_valid_1_entries'] > 0)]
print(f"Number of patients with valid panic events (1 days): {valid_panic_events_patients.shape[0]} / {n_valid_patients_1}")
del valid_panic_events_patients

In [ ]:
patients_with_panic = patient_analysis_data[patient_analysis_data['n_panic'] > 0]
print(f"Number of patients with panic events: {patients_with_panic.shape[0]} / {len(pre_data_ids)}")

### Patient-level Data Group Analysis

In [ ]:
plot_histogram_of_counts(patient_analysis_data['n_dailylog'], title='Number of Daily Log Entries per Patient',
						 xlabel='Number of Daily Log Entries', ylabel='Number of Patients', bins_step=20, exclude_zero=True)
print(f"Number of patients with no daily log entries: {patient_analysis_data[patient_analysis_data['n_dailylog'] == 0].shape[0]} / {len(pre_data_ids)}")
plot_histogram_of_counts(patient_analysis_data['n_lifelog'], title='Number of Life Log Entries per Patient',
						 xlabel='Number of Life Log Entries', ylabel='Number of Patients', bins_step=20, exclude_zero=True)
print(f"Number of patients with no life log entries: {patient_analysis_data[patient_analysis_data['n_lifelog'] == 0].shape[0]} / {len(pre_data_ids)}")
plot_histogram_of_counts(patient_analysis_data['n_questionnaire'], title='Number of Questionnaire Entries per Patient',
						 xlabel='Number of Questionnaire Entries', ylabel='Number of Patients', bins_step=20, exclude_zero=True)
print(f"Number of patients with no questionnaire entries: {patient_analysis_data[patient_analysis_data['n_questionnaire'] == 0].shape[0]} / {len(pre_data_ids)}")

In [ ]:
metadata_panic = metadata[metadata['panic_label'] == 1].copy()
print(f"Number of panic events in preprocessed data: {metadata_panic.shape[0]}")
agg_matrix = [
	('valid_entries_3', 'valid_entry_3', 'sum'),
	('valid_entries_2', 'valid_entry_2', 'sum'),
	('valid_entries_1', 'valid_entry_1', 'sum'),
]
agg_metadata_panic = aggregate_by_column(metadata_panic, 'ID', agg_matrix)

print(f"Number of valid panic entries (valid_entry_3): {agg_metadata_panic['valid_entries_3'].sum()}")
print(f"Number of valid panic entries (valid_entry_2): {agg_metadata_panic['valid_entries_2'].sum()}")
print(f"Number of valid panic entries (valid_entry_1): {agg_metadata_panic['valid_entries_1'].sum()}")
print("--------------------------------------------------------")
print(f"Number of patients with valid panic entries (valid_entry_3): {agg_metadata_panic[agg_metadata_panic['valid_entries_3'] > 0].shape[0]}")
print(f"Number of patients with valid panic entries (valid_entry_2): {agg_metadata_panic[agg_metadata_panic['valid_entries_2'] > 0].shape[0]}")
print(f"Number of patients with valid panic entries (valid_entry_1): {agg_metadata_panic[agg_metadata_panic['valid_entries_1'] > 0].shape[0]}")